In [3]:
import json
from preprocessor.preprocessor_pytorch import Preprocessor
from model.model_pytorch import Model
from postprocessor.postprocessor import Postprocesser
from evaluator.evaluator_pytorch import Evaluator
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import os
import time
import pickle

class ReversePrediction():
    def set_seed(self, seed_value):
        np.random.seed(seed_value)
        tf.random.set_seed(seed_value)
        random.seed(seed_value)
        torch.manual_seed(seed_value)

    def run(self, params):
        self.set_seed(42)
        preprocessor = Preprocessor(params)
        X_train, y_train, X_val, y_val, X_test, y_test, test_dates, X_newest, x_newest_date, y_date, test_dataset = preprocessor.get_multiple_data()

        start_time = time.time()
        model_wrapper = Model(params=params)
        model, history, y_preds, online_history = \
            model_wrapper.run(X_train, y_train, X_test, y_test, X_val, y_val)
        end_time = time.time()
        execution_time = end_time - start_time

        y_preds = torch.tensor(y_preds, dtype=torch.float32)

        y_pred_newest = model.forward(X_newest)
        y_pred_newest = torch.tensor(y_pred_newest, dtype=torch.float32)

        evaluator = Evaluator(params)
        results = evaluator.get_results(y_train, y_val, y_test, y_preds, test_dataset,
                                        test_dates, history, online_history,
                                        show=False)
        results.update({'execution_time': execution_time})

        results_json = json.dumps(results, indent=4)
        with open(params.get('summary_save_path'), 'w') as f:
            f.write(results_json)
            
        return results

In [4]:
open('progress.txt', 'w').close()
open('log.txt', 'w').close()
root_path = 'DNN_Projects'
for floder in tqdm.tqdm(os.listdir(root_path), file=open('progress.txt', 'a')):
    first_path = os.path.join(root_path, floder)
    for subfloder in tqdm.tqdm(os.listdir(first_path), file=open('progress.txt', 'a')):
        second_path = os.path.join(first_path, subfloder)
        print(second_path, file=open('progress.txt', 'a'))
        params = json.load(open(os.path.join(second_path, 'parameters.json'), 'r'))
        reversePrediction = ReversePrediction()
        reversePrediction.set_seed(42)
        try: 
            model_summary, trend_confusion_matrix_info, reversed_trend_confusion_matrix_info, signal_confusion_matrix_info, \
            roc_auc, pred_days_difference_results, pred_days_difference_abs_mean, pred_in_advance, backtesting_report, trade_summary, execution_time = reversePrediction.run(params)
            trend_confusion_matrix_info = trend_confusion_matrix_info.to_json()
            reversed_trend_confusion_matrix_info = reversed_trend_confusion_matrix_info.to_json()
            signal_confusion_matrix_info = signal_confusion_matrix_info.to_json()
            roc_auc_json = json.dumps(roc_auc)
            model_summary_json = json.dumps(model_summary)
            backtesting_report_json = json.dumps(backtesting_report)
            pred_days_difference_results_json = pred_days_difference_results.to_json()
            pred_days_difference_abs_mean_json = json.dumps(pred_days_difference_abs_mean)
            pred_in_advance_json = json.dumps(pred_in_advance)
            trade_summary_json = json.dumps(trade_summary)
            execution_time_json = json.dumps(execution_time)

            response = {
                'msg': 'Received!',
                'usingData': params,
                'model_summary': model_summary_json,
                'trend_confusion_matrix_info': trend_confusion_matrix_info,
                'reversed_trend_confusion_matrix_info': reversed_trend_confusion_matrix_info,
                'signal_confusion_matrix_info': signal_confusion_matrix_info,
                'roc_auc': roc_auc_json,
                'pred_days_difference_results': pred_days_difference_results_json,
                'pred_days_difference_abs_mean': pred_days_difference_abs_mean_json,
                'pred_in_advance': pred_in_advance_json,
                'backtesting_report': backtesting_report_json,
                'trade_summary': trade_summary_json,
                'execution_time': execution_time_json,
            }
            
            with open(params['save_path']['summary_save_path'], 'w') as f:
                json.dump(response, f)
                
            # response
            print('done', file=open('progress.txt', 'a'))
        except Exception as e:
            print(e, file=open('progress.txt', 'a'))
            continue


[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:148: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 0
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:145: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 1
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:152: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instea

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:148: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 0
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:145: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 1
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:152: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instea

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:148: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 0
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:145: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 1
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:155: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instea

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:148: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 0
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:145: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[prev_idx:idx, 'Trend'] = 1
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\preprocessor\preprocessor_pytorch.py:152: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instea

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Training set shape: torch.Size([216, 64, 30])
Validation set shape: torch.Size([7, 64, 30])
Test set shape: torch.Size([45, 64, 30])








  1%|          | 10/1000 [00:00<01:20, 12.25it/s]
C:\Users\e4903\AppData\Local\Temp\ipykernel_18584\3731148982.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred_newest = torch.tensor(y_pred_newest, dtype=torch.float32)
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\evaluator\evaluator_pytorch.py:287: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  buy_signals = trade_signals.loc[x_start:x_stop][(
c:\Users\e4903\Desktop\code\2023_hw\web\saferTrader\stock_project\revpred\lib\reversePrediction\evaluator\evaluator_pytorch.py:296: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sell_signals = trade_signals.loc[x_start:x_stop][(
No artists with labels found to put in legend.  Note that artists whose label start with an 